In [2]:
# packages
import numpy as np
import pandas as pd
import scipy.stats as sc
import matplotlib.pyplot as plt

In [89]:
# data loading
d_loc = 'csv_data/'
ytw = pd.read_csv(d_loc + 'ytw.csv')
ytw['ytw'] = ytw['ytw'].str.rstrip('%').astype('float') / 100.0
ytw['date'] = pd.to_datetime(ytw['date'])
ytm = pd.read_csv(d_loc + 'ytm.csv')
ytm['ytm'] = ytm['ytm'].str.rstrip('%').astype('float') / 100.0;
ytm['date'] = pd.to_datetime(ytm['date'])
expos = pd.read_csv(d_loc + "expos.csv")
expos['date'] = pd.to_datetime(expos['date'])
uso = pd.read_csv(d_loc + 'uso.csv')
uso['trade_date'] = pd.to_datetime(uso['trade_date'])
uso['expirDate'] = pd.to_datetime(uso['expirDate'])
monthly_expos = expos[expos.expo_type == 'Monthly']
max_date_back = max([
    min(uso_stock.index), 
    min(monthly_expos['date']), 
    min(ytm['date']), 
    min(ytw['date'])])

uso = uso[uso["trade_date"] >  max_date_back]

In [108]:
uso.head(), max_date_back

(        stkPx trade_date  expirDate  strike  cOi    pOi  cValue  pValue  \
 210315  42.07 2010-04-05 2010-04-16    18.0   50   1043   24.07     0.0   
 210316  42.07 2010-04-05 2010-04-16    19.0   50   2407   23.07     0.0   
 210317  42.07 2010-04-05 2010-04-16    20.0   10   2292   22.07     0.0   
 210318  42.07 2010-04-05 2010-04-16    21.0    0  25400   21.07     0.0   
 210319  42.07 2010-04-05 2010-04-16    22.0    0  21096   20.07     0.0   
 
         delta  gamma  
 210315    1.0    0.0  
 210316    1.0    0.0  
 210317    1.0    0.0  
 210318    1.0    0.0  
 210319    1.0    0.0  , Timestamp('2010-04-01 00:00:00'))

In [106]:
#### USO Sorting
# putting into one frame
# pd.merge
# pd.shift (for advancing)
uso_stock = uso[['trade_date', 'stkPx']].groupby('trade_date').agg(np.mean)
uso_expos = monthly_expos.merge(uso, left_on='date', right_on='expirDate')
uso_expos = uso_expos.drop(columns=['date', 'expo_type'])
uso_trading = monthly_expos.merge(uso_expos, left_on='date', right_on='trade_date')
uso_trading = uso_trading.sort_values(['trade_date', 'expirDate', 'strike'])

In [115]:
min(np.unique(uso_trading.trade_date)), min(np.unique(uso.trade_date)), min(np.unique(monthly_expos.date))

(numpy.datetime64('2010-12-17T00:00:00.000000000'),
 numpy.datetime64('2010-04-05T00:00:00.000000000'),
 numpy.datetime64('2007-01-20T00:00:00.000000000'))

In [116]:
# building a model on linear regression time series 
# https://towardsdatascience.com/prediction-task-with-multivariate-timeseries-and-var-model-47003f629f9

### Plotting YTM & YTW
This is to visualize the estimated cost of a goverment paying the TEY of the S&P IG Energy Sector YTM & YTW and buying the 20 delta USO put for two months out.

In [117]:
max_date_back

Timestamp('2010-04-01 00:00:00')

In [131]:
delta = .2
put_delta = 1 - delta
first_trade, trade_pass = True, True
for trade_date, chain in uso_trading.groupby('trade_date'):
    first_trade = True
    for expirDate, expo_chain in chain.groupby('expirDate'):
        if first_trade: 
            first_trade = False
            trade_pass = False
            continue
        elif trade_pass:
            break
        else:
            right_delta = expo_chain.iloc[(expo_chain['delta'] - put_delta).abs().argsort()[:1]]
            right_cols = right_delta[['trade_date', 'expirDate', 'pValue', 'strike', 'delta']]
            print(right_cols)
            trade_pass = True

   trade_date  expirDate  pValue  strike   delta
47 2010-12-17 2011-01-21    0.38    35.0  0.8112
    trade_date  expirDate  pValue  strike   delta
283 2011-01-21 2011-02-18     0.3    35.0  0.8204
    trade_date  expirDate  pValue  strike   delta
468 2011-02-18 2011-03-18    0.27    34.0  0.8241
    trade_date  expirDate  pValue  strike   delta
675 2011-03-18 2011-04-15     0.6    38.0  0.7754
     trade_date  expirDate  pValue  strike  delta
1002 2011-04-15 2011-05-20    0.53    41.0  0.777
     trade_date  expirDate  pValue  strike   delta
1282 2011-05-20 2011-06-17    0.44    36.0  0.8193
     trade_date  expirDate  pValue  strike   delta
1567 2011-06-17 2011-07-15    0.49    34.0  0.7822
     trade_date  expirDate  pValue  strike   delta
1857 2011-07-15 2011-08-19    0.41    35.0  0.8153
     trade_date  expirDate  pValue  strike   delta
2121 2011-08-19 2011-09-16    0.59    28.0  0.8164
     trade_date  expirDate  pValue  strike   delta
2410 2011-09-16 2011-10-21    0.54    30.0 

      trade_date  expirDate  pValue  strike   delta
27362 2018-04-20 2018-05-18    0.13    13.0  0.7877
      trade_date  expirDate  pValue  strike   delta
27560 2018-05-18 2018-06-15    0.07    13.5  0.8545
      trade_date  expirDate  pValue  strike   delta
27724 2018-06-15 2018-07-20     0.1    12.0  0.8416
      trade_date  expirDate  pValue  strike   delta
27882 2018-07-20 2018-08-17    0.14    13.5  0.7678
      trade_date  expirDate  pValue  strike   delta
28050 2018-08-17 2018-09-21    0.18    13.0  0.7507
      trade_date  expirDate  pValue  strike   delta
28196 2018-09-21 2018-10-19     0.1    14.0  0.8281
      trade_date  expirDate  pValue  strike   delta
28368 2018-10-19 2018-11-16    0.17    14.0  0.7571
      trade_date  expirDate  pValue  strike   delta
28566 2018-11-16 2018-12-21    0.22    11.0  0.7745
      trade_date  expirDate  pValue  strike   delta
28802 2018-12-21 2019-01-18    0.17     8.5  0.8059
      trade_date  expirDate  pValue  strike   delta
29062 2019-0

In [132]:
wrangler = OptionsWrangler(data=options_data)
## HAS A SUBCLASS: OPTION
    ### it's essentially a row of a dataframe, except it grabs the daily data for itself 
    ### it's the string of the bucket, so to speak
    Option((delta or strike) and trade_date and expo, months_out=None, weeks_out=None,)
# returns the the expiration for an option
wrangler.get_expo_chain(expo_chain, trade_date) # throws not valid expo, not in dataset (before, after)
# returns the the expiration for an option
wrangler.get_option_expo(option)
wrangler.get_option_history(strike, expo, dtarting_trade_date) # throws no such option, not in dataset
wrangler.get_days_historical_chain() # throws not in data set
wrangler.get_imp_vol(expo, trade_date, strike)
wrangler.get_call_value(expo, trade_date, strike) # throws not in dataset 
wrangler.get_put_value(expo, trade_date, strike) # throws not in dataset
wrangler.get_market(expo, trade_date, strike) # throws not in dataset 

IndentationError: unexpected indent (<ipython-input-132-56e446ddd9c3>, line 5)